
# Update recipes

Below are some ad-hoc code chunks to assist me in mass-updating conda-forge R recipes

In [27]:
import os
import sys
import time

In [2]:
# Authenticate with GitHub

import github

gh_token = os.environ['GH_TOKEN']
gh = github.Github(gh_token)
gh_me = gh.get_user()
print("Authenticated as %s"%(gh_me.login))

Authenticated as jdblischak


In [3]:
# Check rate limit
gh.get_rate_limit()

RateLimit(core=Rate(reset=2021-12-31 02:39:54, remaining=4999, limit=5000))

In [6]:
# Search
# https://pygithub.readthedocs.io/en/latest/github.html?highlight=search#github.MainClass.Github.search_code
results = gh.search_code(query="org:conda-forge jdblischak language:YAML GPL-2 0-only")
results.totalCount

11

In [7]:
# Repositories to keep
skip = ['r-feedstock',
        'r-base-feedstock',
        'r-git2r-feedstock',
        'r-knitr-feedstock',
        'r-rmarkdown-feedstock',
        'r-workflowr-feedstock', 'r-reshape-feedstock']

In [8]:
# Convert search results to dict of repository objects
dict_repo = {}
for r in results:
    repo = r.repository
    name_repo = os.path.basename(repo.full_name)
    if name_repo in skip:
        continue
    if name_repo[:2] != "r-":
        continue
    dict_repo[name_repo] = repo
print(len(dict_repo))

11


In [9]:
# Save list of packages to generate with helper script
dir_helper = "../../conda_r_skeleton_helper"
if not os.path.exists(dir_helper):
    sys.stderr.write("Need to clone helper repo")
    sys.exit(1)
file_packages = dir_helper + "/packages.txt"
handle_packages = open(file_packages, "w")
for name_temp in dict_repo.keys():
    handle_packages.write(name_temp[:-10] + "\n")
handle_packages.close()

In [10]:
%%bash -s {file_packages}

wc -l $1

11 ../../conda_r_skeleton_helper/packages.txt


In [11]:
# Fork repositories
for repo in dict_repo.values():
    print(repo.full_name)
    gh_me.create_fork(repo)

conda-forge/r-kriging-feedstock
conda-forge/r-locfdr-feedstock
conda-forge/r-genkern-feedstock
conda-forge/r-genalg-feedstock
conda-forge/r-metap-feedstock
conda-forge/r-signal-feedstock
conda-forge/r-gpseq-feedstock
conda-forge/r-tractor.base-feedstock
conda-forge/r-diffr-feedstock
conda-forge/r-dunn.test-feedstock
conda-forge/r-robustrankaggreg-feedstock


In [12]:
%%bash -s {file_packages}

# Clone repositories

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  if [ ! -d "$localdir" ]
  then
    git clone --quiet git@github.com:jdblischak/${pkg}-feedstock.git "$localdir"
  fi
done

r-kriging
r-locfdr
r-genkern
r-genalg
r-metap
r-signal
r-gpseq
r-tractor.base
r-diffr
r-dunn.test
r-robustrankaggreg


In [13]:
%%bash -s {dir_helper}

# Generate new recipes

# Had to manually set `--allow-archived` flag to regenerate old recipes for
# archived packages

cd "$1"
Rscript run.R

Processing r-kriging
Adding in variants from internal_defaults
Parsing input package r-kriging:
.. name: kriging location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-kriging
Making/refreshing recipe for kriging
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-kriging_1.1_bc20a08af3.tar.gz
Writing recipe for kriging
--dirty flag and --keep-old-work not specified. Removing build/test folder after successful build/test.

Processing r-locfdr
Adding in variants from internal_defaults
Parsing input package r-locfdr:
.. name: locfdr location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-locfdr
Making/refreshing recipe for locfdr
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-locfdr_1.1-8_42d6e12593.tar.gz
Writing recipe for locfdr
--dirty flag and --keep-old-work not spe

INFO:conda_build.variants:Adding in variants from internal_defaults
/home/jdblischak/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/jdblischak/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
No hash (md5, sha1, sha256) provided for source-kriging_1.1.tar.gz.  Source download forced.  Add hash to recipe to use source cache.
INFO:conda_build.config:--dirty flag and --keep-old-work not specified. Removing build/test folder after successful build/test.

INFO:cond

In [20]:
# Get maintainers to add back

dict_maintainers = {}

conda_forge_r = {
    "conda-forge/r",
    "johanneskoester",
    "bgruening",
    "daler",
    "jdblischak",
    "cbrueffer",
    "dbast",
    "dpryan79",
    # typo from r-diffr-feedstock'
    "Johanneskoester",
    "Bgruening",
    "Daler",
    "Jdblischak",
    "Cbrueffer",
    "Dbast",
    "Dpryan79"
}

for name_temp in dict_repo.keys():
    file_codeowners = "/tmp/" + name_temp + "/.github/CODEOWNERS"
    handle_codeowners = open(file_codeowners, "r")
    codeowners = handle_codeowners.readlines()
    handle_codeowners.close()
    codeowners = codeowners[0][2:].replace("@", "").split(" ")
    maintainers = set(codeowners).difference(conda_forge_r)
    dict_maintainers[name_temp] = maintainers

In [15]:
# Increment original build numbers

dict_builds = {}

for name_temp in dict_repo.keys():
    file_meta = "/tmp/" + name_temp + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    for line in handle_meta:
        if line.strip()[:6] == "number":
            build_num = int(line.strip().split(" ")[1])
            build_num = build_num + 1
            dict_builds[name_temp] = build_num
    handle_meta.close()

In [16]:
dict_builds

{'r-kriging-feedstock': 1004,
 'r-locfdr-feedstock': 1003,
 'r-genkern-feedstock': 1005,
 'r-genalg-feedstock': 1004,
 'r-metap-feedstock': 3,
 'r-signal-feedstock': 1,
 'r-gpseq-feedstock': 1003,
 'r-tractor.base-feedstock': 1,
 'r-diffr-feedstock': 1004,
 'r-dunn.test-feedstock': 1003,
 'r-robustrankaggreg-feedstock': 1004}

In [21]:
%%bash -s {dir_helper}

# Copy updated recipes to cloned repositories

pkgs="$1/packages.txt"
for pkg in `cat $pkgs`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  # For some reason globbing doesn't work when run from within the notebook
  cp "$1/$pkg/meta.yaml" "$localdir/recipe/"
  cp "$1/$pkg/build.sh" "$localdir/recipe/"
  cp "$1/$pkg/bld.bat" "$localdir/recipe/"
done

r-kriging
r-locfdr
r-genkern
r-genalg
r-metap
r-signal
r-gpseq
r-tractor.base
r-diffr
r-dunn.test
r-robustrankaggreg


In [22]:
# Add back maintainers, bump version number, and fix license

for feedstock in dict_maintainers.keys():
    maintainers = dict_maintainers[feedstock]
    print(feedstock)
    file_meta = "/tmp/" + feedstock + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    meta = handle_meta.readlines()
    handle_meta.close()
    maintainers_formatted = ["    - " + id + "\n" for id in maintainers]
    lines_out = meta
    for i in range(len(meta)):
        line = meta[i]
        if line.rstrip() == "    - conda-forge/r" and (len(maintainers)) != 0:
            lines_out = meta[:i+1] + maintainers_formatted + meta[i+1:]
    for i in range(len(lines_out)):
        line = lines_out[i]
        if line.rstrip() == "  number: 0":
            lines_out[i] = "  number: %d\n"%(dict_builds[feedstock])
    for i in range(len(lines_out)):
        line = lines_out[i]
        if line.rstrip() == "  license: GPL-2.0-or-later":
            lines_out[i] = "  license: GPL-2.0-only\n"
    handle_meta = open(file_meta, "w")
    handle_meta.write("".join(lines_out))
    handle_meta.close()

r-kriging-feedstock
r-locfdr-feedstock
r-genkern-feedstock
r-genalg-feedstock
r-metap-feedstock
r-signal-feedstock
r-gpseq-feedstock
r-tractor.base-feedstock
r-diffr-feedstock
r-dunn.test-feedstock
r-robustrankaggreg-feedstock


In [23]:
%%bash -s {file_packages}

# Commit updated recipes to branch "update-recipe"

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git checkout -b "update-recipe"
  git add recipe/
  git commit -m "Update recipe"
done

r-kriging
[update-recipe 0b45f2d] Update recipe
 3 files changed, 34 insertions(+), 23 deletions(-)
r-locfdr
[update-recipe 384d51d] Update recipe
 3 files changed, 30 insertions(+), 28 deletions(-)
 rewrite recipe/build.sh (97%)
r-genkern
[update-recipe 522e76d] Update recipe
 3 files changed, 37 insertions(+), 24 deletions(-)
r-genalg
[update-recipe 57d55b3] Update recipe
 3 files changed, 31 insertions(+), 29 deletions(-)
r-metap
[update-recipe 6cce244] Update recipe
 3 files changed, 44 insertions(+), 29 deletions(-)
r-signal
[update-recipe eb9e90c] Update recipe
 3 files changed, 43 insertions(+), 25 deletions(-)
r-gpseq
[update-recipe bfac01b] Update recipe
 3 files changed, 33 insertions(+), 29 deletions(-)
r-tractor.base
[update-recipe bed1c1b] Update recipe
 3 files changed, 43 insertions(+), 30 deletions(-)
 rewrite recipe/build.sh (97%)
r-diffr
[update-recipe 2968826] Update recipe
 3 files changed, 39 insertions(+), 30 deletions(-)
r-dunn.test
[update-recipe a4963b5] Update

Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'


In [24]:
%%bash -s {file_packages}

# Rerender feedstock

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  conda-smithy rerender -c auto
done

r-kriging
Adding in variants from internal_defaults
Adding in variants from /tmp/tmpclb7nrx7/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe a770295] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2021.12.30.13.14.56
 17 files changed, 67 insertions(+), 54 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-locfdr
Adding in variants from internal_defaults
Adding in variants from /tmp/tmpaalfbvmg/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe fad2322] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2021.12.30.13.14.56
 10 files changed, 38 insertions(+), 41 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-genkern
Adding in variants from internal_defaults
Adding in variants from /tmp/tmps3knfwl0/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe b676f8d] MNT: Re-rendered with cond

INFO:conda_smithy.configure_feedstock:Downloading conda-forge-pinning-2021.12.30.13.14.56
INFO:conda_smithy.configure_feedstock:Extracting conda-forge-pinning to /tmp/tmpclb7nrx7
INFO:conda_smithy.configure_feedstock:r410.yaml is closed now. Removing
Setting build platform. This is only useful when pretending to be on another platform, such as for rendering necessary dependencies on a non-native platform. I trust that you know what you're doing.
Setting build arch. This is only useful when pretending to be on another arch, such as for rendering necessary dependencies on a non-native arch. I trust that you know what you're doing.
No numpy version specified in conda_build_config.yaml.  Falling back to default numpy value of 1.16
INFO:conda_build.variants:Adding in variants from internal_defaults
INFO:conda_build.variants:Adding in variants from /tmp/tmpclb7nrx7/conda_build_config.yaml
INFO:conda_build.variants:Adding in variants from argument_variants
/home/jdblischak/miniconda3/lib/pyth

In [25]:
%%bash -s {file_packages}

# Push to GitHub

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git push -f origin update-recipe
done

r-kriging
r-locfdr
r-genkern
r-genalg
r-metap
r-signal
r-gpseq
r-tractor.base
r-diffr
r-dunn.test
r-robustrankaggreg


remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-kriging-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-kriging-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-locfdr-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-locfdr-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-genkern-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-genkern-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.co

In [26]:
# Open pull requests
# https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html?highlight=create_pull#github.Repository.Repository.create_pull
# https://docs.github.com/en/rest/reference/pulls#create-a-pull-request
# https://pygithub.readthedocs.io/en/latest/examples/PullRequest.html?highlight=create%20a%20pull%20request#create-a-new-pull-request
feedstocks = list(dict_repo.keys())
# feedstocks_completed = ["conda-forge/" + stock for stock in feedstocks]
for repo in dict_repo.values():
    # if repo.full_name in feedstocks_completed:
    #     continue
    print(repo.full_name)
    body = """Checklist
* [x] Used a [personal fork of the feedstock to propose changes](https://conda-forge.org/docs/maintainer/updating_pkgs.html#forking-and-pull-requests)
* [x] Bumped the build number (if the version is unchanged)
* [ ] Reset the build number to `0` (if the version changed)
* [x] [Re-rendered]( https://conda-forge.org/docs/maintainer/updating_pkgs.html#rerendering-feedstocks ) with the latest `conda-smithy` (Use the phrase <code>@<space/>conda-forge-admin, please rerender</code> in a comment in this PR for automated rerendering)
* [x] Ensured the license file is being packaged.

* I already rerendered locally
"""
    repo.create_pull(title="Update recipe", body=body, base="master", head="jdblischak:update-recipe")
    time.sleep(1)

conda-forge/r-kriging-feedstock
conda-forge/r-locfdr-feedstock
conda-forge/r-genkern-feedstock
conda-forge/r-genalg-feedstock
conda-forge/r-metap-feedstock
conda-forge/r-signal-feedstock
conda-forge/r-gpseq-feedstock
conda-forge/r-tractor.base-feedstock
conda-forge/r-diffr-feedstock
conda-forge/r-dunn.test-feedstock
conda-forge/r-robustrankaggreg-feedstock


GithubException: 403 {"message": "You have exceeded a secondary rate limit and have been temporarily blocked from content creation. Please retry your request again later.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#secondary-rate-limits"}